In [ ]:
# imports
import numpy as np
from scipy import stats
from scipy.stats import norm


In [ ]:
unit_cost = 10.90
unit_price = 24.00
ave_demand = 32000
std_demand = 11000

In [ ]:
def calculate_profit(cost, price, order, demand):
    # outcome 1: didn't order enough
    if order < demand:
        amount = order
    # outcome 2: less demand than expected
    else:
        amount = demand
    return (unit_price * amount) - (unit_cost * order)


In [ ]:
my_profit = calculate_profit(10.90, 24, 25000, 4000)
print("The profit/(loss): {}".format(my_profit)) # 177.000 in Excel (due to rounding)

In [ ]:
# using scipy
my_prob2 = norm.cdf(8000, ave_demand, std_demand)
print("The probability: {}".format(my_prob2)) # 1.5% in Excel

In [ ]:
# using simulation

# let's take a million samples out of the Normal distribution: samples
samples = np.random.normal(ave_demand, std_demand, size=1000000) # samples is a list of possible demands

# Compute the fraction that are less than or equal to 8000 units: prob
my_prob = np.sum(samples <= 8000)/ len(samples)
print("The probability with simulation: {}".format(my_prob)) # roughly 1.5% again, but bit different

In [ ]:
# shortage cost for now
shortage_cost = unit_price - unit_cost
# excess cost for now
excess_cost = unit_cost
# the critical ratio captures the trade-off
critical_ratio = shortage_cost / (excess_cost + shortage_cost)

In [ ]:
# find Q* where the probability of my demand being less or equal to order equals CR
my_q1 = np.ceil(norm.ppf(critical_ratio, ave_demand, std_demand))
print("The ideal order size: {}".format(my_q1)) # should give 33,267

In [ ]:
# new variable
salvage_price = 7.00

# shortage cost for now
shortage_cost = unit_price - unit_cost
# excess cost for now
excess_cost_salvage = unit_cost - salvage_price
# the critical ratio captures the trade-off
critical_ratio_salvage = shortage_cost / (excess_cost_salvage + shortage_cost)

In [ ]:
# find Q* where the probability of my demand being less or equal to order equals CR
my_q2 = np.ceil(norm.ppf(critical_ratio_salvage, ave_demand, std_demand))
print("The ideal order size with salvage value: {}".format(my_q2)) # should give 40,149

In [ ]:
def calculate_expected_units(order_size, mu, sigma):
    k = (order_size - mu)/sigma
    gk = norm.pdf(k, 0, 1) - (k * norm.sf(k))
    exp_us = gk * sigma # this np.ceil needs to be fixed !!!!
    return (exp_us)

In [ ]:
example = calculate_expected_units(190, 160, 45)
print("The expected unit for normal example is: {}.".format(example))

In [ ]:
def calculate_expected_profits(price, salvage, cost, penalty, order_size, mu, sigma):
    exp_us = calculate_expected_units(order_size, mu, sigma)
    return (price-salvage)*mu - ((cost-salvage)*order_size) - ((price-salvage+penalty)*exp_us)

In [ ]:
# solving CASE 1:
salvage_value1 = 0
exp_us1 = np.ceil(calculate_expected_units(my_q1, ave_demand, std_demand))
my_exp_prof1 = np.ceil(calculate_expected_profits(unit_price, salvage_value1, \
                unit_cost, 0, my_q1, ave_demand, std_demand))
print("The expected profit without salvage value: {}".format(my_exp_prof1)) 
# $314,550 when using the standard normal table ($26 difference)

In [ ]:
# Solving CASE 2:
salvage_value2 = 7.00
exp_us2 = np.ceil(calculate_expected_units(my_q2, ave_demand, std_demand))
my_exp_prof2 = np.ceil(calculate_expected_profits(unit_price, salvage_value2, \
                unit_cost, 0, my_q2, ave_demand, std_demand))
print("The expected profit with salvage value: {}".format(my_exp_prof2))
# $362,514 when using the standard normal table ($14 difference)

In [ ]:
# repeating from marginal cost with salvage

# shortage cost: price - cost
shortage_cost = unit_price - unit_cost
# excess cost: cost - salvage
excess_cost_salvage = unit_cost - salvage_price
# the critical ratio captures the trade-off
critical_ratio_salvage = shortage_cost / (excess_cost_salvage + shortage_cost)
print("The ideal order size with salvage value is still: {}".format(my_q2)) # should give 40,149
print("The expected profit for the retailer with salvage value is: {}".format(my_exp_prof2))

In [ ]:
# new variable: the manufacturer's cost
base_cost = 2.90

man_exp_prof1 = my_q2 * (unit_cost - base_cost)
print("The ideal order size is: {}".format(my_q2))
print("The retailer's profit is: {}".format(my_exp_prof2))
print("The manufacturer's profit is: {}".format(man_exp_prof1))
print("Total channel profit is: {}".format(my_exp_prof2 + man_exp_prof1))

In [ ]:
# shortage cost for now
shortage_cost_channel = unit_price - base_cost
# excess cost for now
excess_cost_salvage = unit_cost - salvage_price
# the critical ratio captures the trade-off
critical_ratio_salvage_channel = shortage_cost_channel / (excess_cost_salvage + shortage_cost_channel)

# finding ideal order size
my_q3 = np.ceil(norm.ppf(critical_ratio_salvage_channel, ave_demand, std_demand))
print("The ideal order size with salvage value for the whole channel is: {}".format(my_q3))

# finding expected profits for the channel
my_exp_prof3 = np.ceil(calculate_expected_profits(unit_price, salvage_value2, \
                unit_cost, 0, my_q3, ave_demand, std_demand))
print("The expected profit of the retailer is lower: {}".format(my_exp_prof3))

# finding manufacturer profits at this level
man_exp_prof2 = my_q3 * (unit_cost - base_cost)
print("Total channel profit is: {}".format(man_exp_prof2+my_exp_prof3))

In [ ]:
# buyback contract
buyback_price = (((unit_price - salvage_price) / (unit_price - base_cost)) * unit_cost) \
                - ((unit_price*(base_cost - salvage_price)) / (unit_price - base_cost))

my_exp_prof4 = np.ceil(calculate_expected_profits(unit_price, buyback_price, \
                unit_cost, 0, my_q3, ave_demand, std_demand))

print("The optimal buyback price is {}".format(round(buyback_price, 2)))
print("The ideal order size with salvage value for the whole channel is: {}".format(my_q3))
print("Total channel profit is: {}".format(man_exp_prof2+my_exp_prof4))
print("The retailer's profit is now: {}".format(my_exp_prof4)) # previously it was $362,500
print("The manufacturer's profit is now: {}".format(man_exp_prof2)) # previously it was $321,192